# Import the Necessary Libraries

In [1]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Bring in the Neighborhood Data

In [3]:
philly2 = pd.read_csv(r'/Users/kylenavroth/Downloads/Philly Zips Lat Long - Sheet1.csv')
philly2

,Zip Code,Latitude,Longitude
0,19019,40.002,-75.118
1,19244,40.002,-75.118
2,19255,40.002,-75.118
3,19109,39.950,-75.164
4,19175,39.991,-75.130
...,...,...,...
86,19124,40.017,-75.088
87,19128,40.039,-75.221
88,19103,39.952,-75.174
89,19141,40.036,-75.145


In [4]:
philly = pd.read_csv(r'/Users/kylenavroth/Downloads/Philly Neighborhoods - Sheet1.csv')
philly

,Neighborhood,Zip Code
0,Rittenhouse Square,19102
1,Avenue of The Arts,19102
2,Rittenhouse Square,19103
3,Avenue of The Arts,19103
4,Logan Square,19103
...,...,...
78,Overbrook Park,19151
79,Rhawnhurst,19152
80,Eastwick,19153
81,Parkwood,19154


In [5]:
print(philly2.shape)
print(philly.shape)

(91, 3)
(83, 2)


In [5]:
philly2.set_index('Zip Code')
philly.set_index('Zip Code')

,Neighborhood
Zip Code,
19102,Rittenhouse Square
19102,Avenue of The Arts
19103,Rittenhouse Square
19103,Avenue of The Arts
19103,Logan Square
...,...
19151,Overbrook Park
19152,Rhawnhurst
19153,Eastwick


In [6]:
df_philly = pd.merge(philly, philly2, on='Zip Code', how='outer')
df_philly.head()

,Neighborhood,Zip Code,Latitude,Longitude
0,Rittenhouse Square,19102,39.953,-75.166
1,Avenue of The Arts,19102,39.953,-75.166
2,Rittenhouse Square,19103,39.952,-75.174
3,Avenue of The Arts,19103,39.952,-75.174
4,Logan Square,19103,39.952,-75.174


In [7]:
df_phl = df_philly.dropna()
df_phl

,Neighborhood,Zip Code,Latitude,Longitude
0,Rittenhouse Square,19102,39.953,-75.166
1,Avenue of The Arts,19102,39.953,-75.166
2,Rittenhouse Square,19103,39.952,-75.174
3,Avenue of The Arts,19103,39.952,-75.174
4,Logan Square,19103,39.952,-75.174
...,...,...,...,...
78,Overbrook Park,19151,39.976,-75.253
79,Rhawnhurst,19152,40.060,-75.048
80,Eastwick,19153,39.899,-75.232
81,Parkwood,19154,40.091,-74.977


In [9]:
df_phl.shape

(83, 4)

In [8]:
philly_df = df_phl.groupby('Zip Code').Neighborhood.agg([('Neighborhood', ', '.join)])
philly_df.head()

,Neighborhood
Zip Code,
19102,"Rittenhouse Square, Avenue of The Arts"
19103,"Rittenhouse Square, Avenue of The Arts, Logan ..."
19104,"University City, Belmont, Parkside, Powelton V..."
19106,"Society Hill, Old City"
19107,"Washington Square West,, Avenue of The Arts, M..."


In [9]:
phl = pd.merge(philly_df, philly2, on='Zip Code', how='outer')
philly = phl.dropna()
philly

,Zip Code,Neighborhood,Latitude,Longitude
0,19102,"Rittenhouse Square, Avenue of The Arts",39.953,-75.166
1,19103,"Rittenhouse Square, Avenue of The Arts, Logan ...",39.952,-75.174
2,19104,"University City, Belmont, Parkside, Powelton V...",39.962,-75.200
3,19106,"Society Hill, Old City",39.951,-75.146
4,19107,"Washington Square West,, Avenue of The Arts, M...",39.952,-75.159
5,19111,"Lawndale, Castor Gardens",40.058,-75.080
6,19112,Navy Yard,39.896,-75.190
7,19114,Torresdale,40.064,-75.002
8,19115,"Bustleton, Somerton",40.093,-75.041
9,19116,"Somerton, Bustleton",40.117,-75.015


In [10]:
address = 'Philadelphia, PA'

geolocator = Nominatim(user_agent="philly_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Philly are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Philly are 39.9527237, -75.1635262.


In [13]:
map_philly = folium.Map(location=[latitude, longitude], zoom_start=15)

map_philly

# Bring in the Foursquare Data

In [11]:
CLIENT_ID = 'PJ12IFKJPMCCL15H3OJCE0XYA0GHLQVIYRL4OUOORYK3BH50' # your Foursquare ID
CLIENT_SECRET = 'BF1HWYCE44JKBEFTBC5R5WVMVE1QBA5W1CXEOAL1VU1FKTL3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PJ12IFKJPMCCL15H3OJCE0XYA0GHLQVIYRL4OUOORYK3BH50
CLIENT_SECRET:BF1HWYCE44JKBEFTBC5R5WVMVE1QBA5W1CXEOAL1VU1FKTL3


In [12]:
neighborhood_latitude = philly.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = philly.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = philly.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rittenhouse Square, Avenue of The Arts are 39.953, -75.166.


In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PJ12IFKJPMCCL15H3OJCE0XYA0GHLQVIYRL4OUOORYK3BH50&client_secret=BF1HWYCE44JKBEFTBC5R5WVMVE1QBA5W1CXEOAL1VU1FKTL3&v=20180605&ll=39.953,-75.166&radius=500&limit=100'

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff9d30a9fc7d574b2aeddd2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Center City West',
  'headerFullLocation': 'Center City West, Philadelphia',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 166,
  'suggestedBounds': {'ne': {'lat': 39.957500004500005,
    'lng': -75.16014066049972},
   'sw': {'lat': 39.9484999955, 'lng': -75.17185933950027}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bde0d566198c9b6c5cc12ff',
       'name': 'Dilworth Park',
       'location': {'address': '1 Penn Sq',
        'crossStreet': 'West Side of City Hall',
        'lat': 39.952771846343104,
      

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-16-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Dilworth Park,Park,39.952772,-75.164723
1,La Colombe Coffee Roasters,Coffee Shop,39.951659,-75.165238
2,One Liberty Observation Deck,Scenic Lookout,39.952740,-75.168068
3,JFK Plaza / Love Park,Plaza,39.954123,-75.165303
4,City Hall Courtyard,Plaza,39.952484,-75.163592


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
philly_venues = getNearbyVenues(names=philly['Neighborhood'],
                                   latitudes=philly['Latitude'],
                                   longitudes=philly['Longitude']
                                  )

Rittenhouse Square, Avenue of The Arts
Rittenhouse Square, Avenue of The Arts, Logan Square, Fitler Square
University City, Belmont, Parkside, Powelton Village
Society Hill, Old City
Washington Square West,, Avenue of The Arts, Midtown Village, Chinatown
Lawndale, Castor Gardens
Navy Yard
Torresdale
Bustleton, Somerton
Somerton, Bustleton
Chestnut Hill
Mount Airy
Logan, Olney, Feltonville
Brewerytown
Yorktown, Old Kensington, Jinogi
Northern Liberties, Loft District
Juniata, Frankford, Feltonville
Fishtown, Kensington
Oak Lane
Manayunk
Roxborough
East Falls
Art Museum, Francisville
Wynnefield
Strawberry Mansion
Fairhill, North Philadelphia
Port Richmond
Tacony
Holmesburg
Bridesburg
West Oak Lane
Walnut Hill/West
Hunting Park
Logan
Elmwood, Southwest Philadelphia
University City
Germantown
Passyunk
Graduate Hospital, Naval Square, Southwest Center City
Queen Village, Bella Vista, Pennsport, Italian Market
Whitman, Pennsport, South Philadelphia
Frankford, Mayfair
Cedarbrook
Overbrook, Ov

In [19]:
# one hot encoding
philly_onehot = pd.get_dummies(philly_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
philly_onehot['Neighborhood'] = philly_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [philly_onehot.columns[-1]] + list(philly_onehot.columns[:-1])
philly_onehot = philly_onehot[fixed_columns]

philly_onehot.head()

,Neighborhood,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Rittenhouse Square, Avenue of The Arts",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rittenhouse Square, Avenue of The Arts",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rittenhouse Square, Avenue of The Arts",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rittenhouse Square, Avenue of The Arts",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rittenhouse Square, Avenue of The Arts",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
philly_grouped = philly_onehot.groupby('Neighborhood').mean().reset_index()
philly_grouped

,Neighborhood,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Art Museum, Francisville",0.0,0.0,0.041667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.041667,0.00,0.00,0.000000,0.000000,0.041667
1,Brewerytown,0.0,0.0,0.000000,0.000000,0.000000,0.153846,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
2,Bridesburg,0.0,0.0,0.062500,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
3,"Bustleton, Somerton",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
4,Cedarbrook,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
5,Chestnut Hill,0.0,0.0,0.096774,0.000000,0.000000,0.000000,0.00,0.064516,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
6,East Falls,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
7,Eastwick,0.0,0.0,0.166667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
8,"Elmwood, Southwest Philadelphia",0.2,0.2,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
9,"Fairhill, North Philadelphia",0.0,0.0,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000


In [22]:
num_top_venues = 5

for hood in philly_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = philly_grouped[philly_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Art Museum, Francisville----
                 venue  freq
0                  Bar  0.08
1   Italian Restaurant  0.08
2  Monument / Landmark  0.04
3         Gourmet Shop  0.04
4            Bookstore  0.04


----Brewerytown----
                venue  freq
0       Deli / Bodega  0.15
1         Art Gallery  0.15
2          Restaurant  0.15
3                Park  0.08
4  Athletics & Sports  0.08


----Bridesburg----
               venue  freq
0        Pizza Place  0.12
1  Convenience Store  0.12
2           Pharmacy  0.12
3                Bar  0.06
4        Coffee Shop  0.06


----Bustleton, Somerton----
                        venue  freq
0                        Food   0.5
1                        Pool   0.5
2                Optical Shop   0.0
3           Mobile Phone Shop   0.0
4  Modern European Restaurant   0.0


----Cedarbrook----
                  venue  freq
0            Shoe Store  0.33
1    Athletics & Sports  0.33
2  Caribbean Restaurant  0.33
3                   ATM  0.00
4  

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Give the User a Visual Display of the Neighborhoods

In [23]:
map_philly = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(philly['Latitude'], philly['Longitude'], philly['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_philly)  
    
map_philly

# Introduce the US Cencus Data 

In [26]:
house_value = pd.read_csv(r'/Users/kylenavroth/Downloads/Cencus Data/Median House Value.csv')
house_value.head()

,geoid,name,B25077001,"B25077001, Error"
0,86000US08030,8030,127300.0,7617.0
1,86000US08063,8063,150500.0,6503.0
2,86000US08065,8065,175200.0,7056.0
3,86000US08075,8075,223700.0,6049.0
4,86000US08077,8077,274700.0,4227.0


In [27]:
house_value.drop(['B25077001, Error'], axis=1, inplace=True)
house_value.head()

,geoid,name,B25077001
0,86000US08030,8030,127300.0
1,86000US08063,8063,150500.0
2,86000US08065,8065,175200.0
3,86000US08075,8075,223700.0
4,86000US08077,8077,274700.0


In [28]:
house_value.rename(columns={'B25077001':'Median Value','name': 'Zip Code'}, inplace=True)
house_value.head()

,geoid,Zip Code,Median Value
0,86000US08030,8030,127300.0
1,86000US08063,8063,150500.0
2,86000US08065,8065,175200.0
3,86000US08075,8075,223700.0
4,86000US08077,8077,274700.0


In [29]:
table1 = pd.merge(phl, house_value, on='Zip Code', how='outer')
table1.head()

,Zip Code,Neighborhood,Latitude,Longitude,geoid,Median Value
0,19102,"Rittenhouse Square, Avenue of The Arts",39.953,-75.166,86000US19102,467600.0
1,19103,"Rittenhouse Square, Avenue of The Arts, Logan ...",39.952,-75.174,86000US19103,506800.0
2,19104,"University City, Belmont, Parkside, Powelton V...",39.962,-75.200,86000US19104,138300.0
3,19106,"Society Hill, Old City",39.951,-75.146,86000US19106,516900.0
4,19107,"Washington Square West,, Avenue of The Arts, M...",39.952,-75.159,86000US19107,382300.0


# Create the Final Table for Analysis

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = philly_grouped['Neighborhood']

for ind in np.arange(philly_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(philly_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted['Median Value'] = table1['Median Value']
neighborhoods_venues_sorted.sort_values(by=['Median Value'])

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Median Value
24,Oak Lane,Convenience Store,Pizza Place,Pharmacy,Intersection,Automotive Shop,Donut Shop,Dive Bar,Farm,Falafel Restaurant,Fabric Shop,54600.0
25,"Overbrook, Overbrook Farms, Overbrook Park",Pizza Place,School,Cosmetics Shop,Pharmacy,Seafood Restaurant,Playground,Fabric Shop,Event Space,English Restaurant,Electronics Store,55800.0
32,"Rittenhouse Square, Avenue of The Arts, Logan ...",American Restaurant,Italian Restaurant,Hotel,Deli / Bodega,Coffee Shop,Bakery,Grocery Store,Clothing Store,Sushi Restaurant,Seafood Restaurant,64400.0
34,"Society Hill, Old City",Italian Restaurant,Historic Site,History Museum,Coffee Shop,Art Gallery,Hotel,Pub,Beer Garden,Gastropub,Ice Cream Shop,82600.0
31,"Rittenhouse Square, Avenue of The Arts",Hotel,Coffee Shop,Bakery,Spa,Salad Place,American Restaurant,Clothing Store,Bar,Yoga Studio,Mediterranean Restaurant,83300.0
26,"Parkwood, Byberry",Breakfast Spot,Pizza Place,Pharmacy,Dog Run,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space,English Restaurant,88400.0
35,"Somerton, Bustleton",Flower Shop,Grocery Store,Shopping Mall,Pizza Place,Beer Store,Beer Garden,Liquor Store,Caucasian Restaurant,Donut Shop,Bank,94300.0
12,Germantown,American Restaurant,Coffee Shop,Breakfast Spot,Cocktail Bar,Mobile Phone Shop,Liquor Store,Southern / Soul Food Restaurant,Bookstore,Donut Shop,Discount Store,99700.0
16,"Juniata, Frankford, Feltonville",Bus Station,Miscellaneous Shop,Shoe Store,Clothing Store,Chinese Restaurant,Metro Station,Bar,Yoga Studio,Drugstore,Farm,104100.0
13,"Graduate Hospital, Naval Square, Southwest Cen...",Breakfast Spot,Bar,Deli / Bodega,Park,Coffee Shop,Café,Mexican Restaurant,Furniture / Home Store,Discount Store,Brewery,106500.0


Here a user can analyze the neighborhoods of Philadelphia by a median home value and see what the most common venues are in that neighborhood. If the user has a specific price range in mind, they can find that area and compare the neighborhoods that are in their range. This will be a good place to start when looking to buy a home in the Philadelphia area. 